# Text generation with an RNN - Character-based language model. 

This tutorial demonstrates how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Read the data

First, look in the text:

In [0]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

In [0]:
# Take a look at the first 250 characters in text
print(text[:250])

In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

In [0]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [0]:
model.summary()

To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [0]:
sampled_indices

Decode these to see the text predicted by this untrained model:

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [0]:
accuracy_from_logits=tf.keras.metrics.SparseCategoricalAccuracy()
model.compile(optimizer='adam', loss=loss, metrics=[accuracy_from_logits])

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=10

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.


![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"ROMEO: "))

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement _curriculum learning_ to help stabilize the model's open-loop output.

We will use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, initialize the RNN state. We do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.



In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [0]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

### Exercise - 31/01/2020
* Add a validation set by splitting the resulting numpy array using the train_test_split function of 'sklearn.model_selection'
* Starting from the original RNN of the notebook, do some hyperparameter tuning to find a better model that improves the performance of your task. 
* Change the pre-processing step to create a word-based language model (predicting the next word from the past sequence of words
* Redo hyperparameter tuning for this task. 
* You can after change the text dataset by taking as .txt file another publicly available text from the classic litterature. 
> Don't take it too long; otherwise the network will take too much time to be trained...

# Kamal Chouhbi



# Create a word-based language model

I'll try to use the last RNN but this time to predict or generate words instead of characters:

## WordCloud : 

I'll try to draw a wordcloud just to get an idea about the most frequent words in our Shakespear's dataset

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

wordcld = pd.Series(text)
# Most frequent words in the data set
from wordcloud import WordCloud 
cloud = WordCloud(max_font_size=40, relative_scaling=.5,margin=2,background_color='white').generate(wordcld.str.cat())
plt.figure(figsize=(15, 8))
plt.imshow(cloud)
plt.axis('off')
plt.show()

Then I'll determine the total number of words and the frequency of each word in the whole of Shakespeare's dataset.

### We need to define a function that cleans up any string we pass


In [0]:
def split_and_clean_line(mytext):
    #input: string
    #output: a python list of lowercase words without punctuation
    
    a = mytext.lower().strip()
    
    # punctuation ==>  '!,.-_~`?@#$%^&*()[]{};:+\"'
    punct = ["'",'"','~','!','--','.',",",'-','?','`','@','#','$','%','^','&','(',')',"[","]",'{','}']
    
    # for each character, check punctuation and remove it
    out = []
    for i in a:
        if i not in punct:
            out.append(i)
            
    # join the rest as a string
    clean_text = ''.join(out).split()
    
    return clean_text

print(split_and_clean_line(text[:100]))
text[:100]

Then we will count the most frequent words:

In [0]:
words={} #Dictionnaire

for i in text.splitlines():
  # split and clean that line, then for each word in the line:
        for w in split_and_clean_line(i):
            if len(w) > 5:
                if w in words:
                    # if in the dictionary already, iterate the count for that key
                    words[w] += 1
                else:
                    # if not, initialize the key for that word
                    words[w] = 1           
print('Number of different words!',len(words.keys()))
I = np.argsort(np.array(list(words.values())))[::-1]
wList = np.array(list(words.keys()))[I]
cList = np.array(list(words.values()))[I]
print('-----')

for i in range(10):
    print(wList[i],cList[i])
    
#return words
    

We need to convert now the text into clean tokens :

In [0]:
import re
def preprocess_sentence(w):
  w = w.lower().strip()
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.rstrip().strip()
  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  return w

In [0]:
preprocess_sentence(text[:100])

In [0]:
def clean_text(mytext):
    # split into tokens by white space
    mytext = preprocess_sentence(mytext)
    tokens = mytext.split()
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens
 
tokens = clean_text(text)

number_of_unique_tokens = len(set(tokens))

In [0]:
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % number_of_unique_tokens)
print('These are the first 200 tokens: %s' % tokens[:200])

We need to check how long the input sequences should be. They need to be long enough to allow the model to learn the context as much as possible for the words to predict. 
There is no correct answer, we can try different sized input sequences.

In [0]:
# I'll choose to begin with 10 as sequence length
sequence_length = 10

# organize into sequences of tokens of input words plus one output word
length = sequence_length + 1

sequences = []
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)

print ('Total Sequences: %d' % len(sequences))
print ('This is the first sequence: {0}'.format(sequences[0]))

In [0]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    corpus = corpus[:7000] ## I chose to take just a part of the text 
    #in order to run the model quickly, we can drop the line if we want
    #to train on the full data
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(sequences)
inp_sequences[:10]

In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
predictors.shape, label.shape

#Training

![alt text](http://www.shivambansal.com/blog/text-lstm/2.png)



## Build the model
I have added total three layers in the model.

- Input Layer : Takes the sequence of words as input
- LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later.
- Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer. It helps in preventing over fitting. (Optional Layer)
- Output Layer : Computes the probability of the best possible next word as output

In [0]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = tf.keras.Sequential([
            tf.keras.layers.Embedding(total_words, 10, input_length=input_len),
    
    # Add Hidden Layer 1 - LSTM Layer
           tf.keras.layers.LSTM(100),
           tf.keras.layers.Dropout(0.4),
    
    # Add Output Layer
            tf.keras.layers.Dense(total_words, activation='softmax')])

    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

In [0]:
lr_model_history=model.fit(predictors, label, epochs=5, verbose=1)

In [0]:
def Plot_loss_acc():
   # Plot the loss function
     fig, ax = plt.subplots(1, 1, figsize=(10,6))
     ax.plot(np.sqrt(lr_model_history.history['loss']), 'r', label='train')
     ax.set_xlabel(r'Epoch', fontsize=20)
     ax.set_ylabel(r'Loss', fontsize=20)
     ax.legend()
     ax.tick_params(labelsize=20)

    # Plot the accuracy
     fig, ax = plt.subplots(1, 1, figsize=(10,6))
     ax.plot(np.sqrt(lr_model_history.history['acc']), 'b', label='train')
     ax.set_xlabel(r'Epoch', fontsize=20)
     ax.set_ylabel(r'Accuracy', fontsize=20)
     ax.legend()
     ax.tick_params(labelsize=20)

In [0]:
Plot_loss_acc()

Maybe we need to increase the number of epochs so as to allow a good training for our model:

In [0]:
lr_model_history=model.fit(predictors, label, epochs=50, verbose=2)

In [0]:
Plot_loss_acc()

## Generating words :

In [0]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [0]:
print ("1. ",generate_text("King", 20, model, max_sequence_len))
print ("2. ",generate_text("The princess", 20, model, max_sequence_len))


## Hyperparameter tuning 
Now I'll try to find a better model that improves the performance of the RNN model

## 1- Change the optimizer : 
- RMSprop

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='RMSProp', metrics=['acc'])
lr_model_history=model.fit(predictors, label, epochs=50, verbose=2)

In [0]:
Plot_loss_acc()

- SGD

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])
lr_model_history=model.fit(predictors, label, epochs=50, verbose=2)

In [0]:
Plot_loss_acc()

So by comparing the 3 optimizers we used before, we should keep 'adam' as an optimizers.

## 2- Size of the LSTM

In [0]:
def create_model2(max_sequence_len, total_words,size):
    input_len = max_sequence_len - 1
    model = tf.keras.Sequential([
            tf.keras.layers.Embedding(total_words, 10, input_length=input_len),
    
    # Add Hidden Layer 1 - LSTM Layer
           tf.keras.layers.LSTM(size),
           tf.keras.layers.Dropout(0.4),
    
    # Add Output Layer
            tf.keras.layers.Dense(total_words, activation='softmax')])

    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])
    
    return model

- Size : 256

In [0]:
model = create_model2(max_sequence_len, total_words,256)
model.summary()

In [0]:
lr_model_history=model.fit(predictors, label, epochs=50, verbose=2)

In [0]:
Plot_loss_acc()

- Size : 512

In [0]:
model = create_model2(max_sequence_len, total_words,512)
model.summary()

In [0]:
lr_model_history=model.fit(predictors, label, epochs=50, verbose=2)

In [0]:
Plot_loss_acc()

So it's better to choose a big size of the LSTM Layer, LSTM(512) gave us the best results.

## 3- Dropout level

- 20%

In [0]:
def create_model3(max_sequence_len, total_words,drop):
    input_len = max_sequence_len - 1
    model = tf.keras.Sequential([
            tf.keras.layers.Embedding(total_words, 10, input_length=input_len),
    
    # Add Hidden Layer 1 - LSTM Layer
           tf.keras.layers.LSTM(512),
           tf.keras.layers.Dropout(drop),
    
    # Add Output Layer
            tf.keras.layers.Dense(total_words, activation='softmax')])

    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])
    
    return model

In [0]:
model = create_model3(max_sequence_len, total_words,0.2)
model.summary()

In [0]:
lr_model_history=model.fit(predictors, label, epochs=50, verbose=2)

In [0]:
Plot_loss_acc()

- 30 %

In [0]:
model = create_model3(max_sequence_len, total_words,0.3)
model.summary()

In [0]:
lr_model_history=model.fit(predictors, label, epochs=50, verbose=2)

In [0]:
Plot_loss_acc()

- 50%

In [0]:
model = create_model3(max_sequence_len, total_words,0.5)
model.summary()

In [0]:
lr_model_history=model.fit(predictors, label, epochs=50, verbose=2)

In [0]:
Plot_loss_acc()

So we'll choose a dropout with **0.3** . 


# Text Generator
Now we'll build & fit again our best model in order to generate some text:

In [0]:
model = create_model3(max_sequence_len, total_words,0.3)
model.summary()

In [0]:
model.fit(predictors, label, epochs=30, verbose=2)

In [0]:
print ("1. ",generate_text("First", 20, model, max_sequence_len))
print ("2. ",generate_text(".", 20, model, max_sequence_len))

# Sum-Up
## Now I think the model can generate more comprehensive words than before. I need to precise that I've trained the model just on 7000 Sequences for the sake of the computational time. To get the optimal performance, we need to train on the total Sequences: 240857 . Go Up and change the corpus size in the function : get_sequence_of_tokens()

# Trying our model on an another dataset
This time we'll try to work with a dataset of Victor Hugo that I have uploaded to my GitHub account

In [0]:
import urllib
content=urllib.request.urlopen('https://raw.githubusercontent.com/chouhbik/RNN---Text-Generation/master/victor_hugo-texts.txt') 
text2=''
for line in content: 
    text2 = text2 + line.decode(encoding='utf-8')

In [0]:
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text2)))

In [0]:
# Take a look at the first 250 characters in text
print(text2[:250])

In [0]:
print(split_and_clean_line(text2[:100]))
text2[:100]

In [0]:
words={} #Dictionnaire

for i in text2.splitlines():
  # split and clean that line, then for each word in the line:
        for w in split_and_clean_line(i):
            if len(w) > 5:
                if w in words:
                    # if in the dictionary already, iterate the count for that key
                    words[w] += 1
                else:
                    # if not, initialize the key for that word
                    words[w] = 1           
print('Number of different words!',len(words.keys()))
I = np.argsort(np.array(list(words.values())))[::-1]
wList = np.array(list(words.keys()))[I]
cList = np.array(list(words.values()))[I]
print('-----')

for i in range(10):
    print(wList[i],cList[i])
    
#return words

In [0]:
preprocess_sentence(text2[:100])

In [0]:
tokens2 = clean_text(text2)

number_of_unique_tokens = len(set(tokens2))
print('Total Tokens: %d' % len(tokens2))
print('Unique Tokens: %d' % number_of_unique_tokens)
print('These are the first 200 tokens: %s' % tokens2[:200])

In [0]:
# I'll choose to begin with 10 as sequence length
sequence_length = 10

# organize into sequences of tokens of input words plus one output word
length = sequence_length + 1

sequences2 = []
for i in range(length, len(tokens2)):
    # select sequence of tokens
    seq = tokens2[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences2.append(line)

print ('Total Sequences: %d' % len(sequences2))
print ('This is the first sequence: {0}'.format(sequences2[0]))

In [0]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences2, total_words2 = get_sequence_of_tokens(sequences2)
inp_sequences2[:10]

In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def generate_padded_sequences2(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = to_categorical(label, num_classes=total_words2)
    return predictors, label, max_sequence_len

predictors2, label2, max_sequence_len = generate_padded_sequences2(inp_sequences2)
predictors2.shape, label2.shape

Building the new model to be adapted to our input embedding:

In [0]:
def create_model5(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = tf.keras.Sequential([
            tf.keras.layers.Embedding(total_words, 10, input_length=input_len),
    
    # Add Hidden Layer 1 - LSTM Layer
           tf.keras.layers.LSTM(512),
           tf.keras.layers.Dropout(0.3),
    
    # Add Output Layer
            tf.keras.layers.Dense(total_words, activation='softmax')])

    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])
    
    return model

model = create_model(max_sequence_len, total_words2)
model.summary()

In [0]:
model.fit(predictors2, label2, epochs=30, verbose=2)

In [0]:
print ("1. ",generate_text("La", 20, model, max_sequence_len))
print ("2. ",generate_text("Il", 20, model, max_sequence_len))

We still can improve our model, anyway it gave us some really good results. :)